<a href="https://colab.research.google.com/github/Mehdi-Lahrach/unsupervised_classification_casablanca/blob/main/unsupervised_classification_KMEANS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://githubtocolab.com/gee-community/geemap/blob/master/examples/notebooks/31_unsupervised_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [1]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 42.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.6 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11135 sha256=ed7d5be9a6f0f9c24f351c850906ca000d9f0d203ec7c5447c8523

# Machine Learning with Earth Engine - Unsupervised Classification

![](https://i.imgur.com/IcBapEx.png)

## Step-by-step tutorial

### Import libraries

In [14]:
import ee
import geemap


### Create an interactive map

In [15]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Add data to the map

In [16]:
# Define a polygon geometry for the Casablanca area
roi = ee.Geometry.Polygon([
    [[-7.7577, 33.4949], [-7.7577, 33.6091], [-7.5367, 33.6091], 
     [-7.5367, 33.4949], [-7.7577, 33.4949]]
])

# Filter the Sentinel-2 MSI image collection by the ROI and date range
image = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(roi)
    .filterDate('2019-01-01', '2019-12-31')
    .sort('CLOUDY_PIXEL_PERCENTAGE')
    .first()
    .select(['B4', 'B3', 'B2'])
)

# Define visualization parameters
vis_params = {'min': 0, 'max': 3000, 'gamma': 1.4}


# Center the map on the ROI and add the Sentinel-2 MSI image
Map.centerObject(roi, 10)
#Map.addLayer(image.clip(roi), vis_params, "Sentinel-2 MSI")
Map.addLayer(image, vis_params, "Sentinel-2 MSI")
Map

Map(bottom=1014.4444923400879, center=[33.55203650002733, -7.647199999999738], controls=(WidgetControl(options…

### Check image properties

In [ ]:
props = geemap.image_props(image)
props.getInfo()

{'AOT_RETRIEVAL_ACCURACY': 0,
 'CLOUDY_PIXEL_PERCENTAGE': 0.062289,
 'CLOUD_COVERAGE_ASSESSMENT': 0.062289,
 'CLOUD_SHADOW_PERCENTAGE': 0.002386,
 'DARK_FEATURES_PERCENTAGE': 0.229502,
 'DATASTRIP_ID': 'S2B_OPER_MSI_L2A_DS_SGS__20190924T134513_S20190924T111931_N02.13',
 'DATATAKE_IDENTIFIER': 'GS2B_20190924T110759_013319_N02.13',
 'DATATAKE_TYPE': 'INS-NOBS',
 'DEGRADED_MSI_DATA_PERCENTAGE': 0,
 'FORMAT_CORRECTNESS': 'PASSED',
 'GENERAL_QUALITY': 'PASSED',
 'GENERATION_TIME': 1569332713000,
 'GEOMETRIC_QUALITY': 'PASSED',
 'GRANULE_ID': 'L2A_T29SPT_A013319_20190924T111931',
 'HIGH_PROBA_CLOUDS_PERCENTAGE': 0.027989,
 'IMAGE_DATE': '2019-09-24',
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B1': 156.588297706,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B10': 158.891172947,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B11': 157.425833779,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B12': 156.696719258,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B2': 162.370787219,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B3': 159.708082292,
 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B4': 15

In [ ]:
props.get('IMAGE_DATE').getInfo()

'2019-09-24'

In [ ]:
props.get('CLOUDY_PIXEL_PERCENTAGE').getInfo()

0.062289

### Make training dataset

There are several ways you can create a region for generating the training dataset.

- Draw a shape (e.g., rectangle) on the map and the use `region = Map.user_roi`
- Define a geometry, such as `region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])`
- Create a buffer zone around a point, such as `region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)`
- If you don't define a region, it will use the image footprint by default

In [ ]:
#region = image.geometry()
region = roi = ee.Geometry.Polygon([
    [[-7.7577, 33.4949], [-7.7577, 33.6091], [-7.5367, 33.6091], 
     [-7.5367, 33.4949], [-7.7577, 33.4949]]
])

#region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
#region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)

In [10]:
# Define a polygon geometry for the Casablanca area
roi = ee.Geometry.Polygon([
    [[-7.7577, 33.4949], [-7.7577, 33.6091], [-7.5367, 33.6091], 
     [-7.5367, 33.4949], [-7.7577, 33.4949]]
])


# Make the training dataset
training = image.sample(
    **{
        'region': roi,
        'scale': 10,
        'numPixels': 6000,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    }
)

# Add the training dataset to the map
Map.addLayer(training, {}, 'training', False)
Map

Map(bottom=105244.0902709961, center=[33.74250969249172, -7.3228641182071765], controls=(WidgetControl(options…

### Train the clusterer

In [11]:
# Instantiate the clusterer and train it.
n_clusters = 15
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

### Classify the image

In [13]:
#Cluster the input using the trained clusterer.
result = image.cluster(clusterer)

#Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map 

Map(bottom=105295.2222442627, center=[33.68409969329066, -7.029064649784032], controls=(WidgetControl(options=…

### Label the clusters

In [ ]:
legend_keys = ['One', 'Two', 'Three', 'Four', 'five','sita']
legend_colors = ['#FF3333', '#FFC733', '#77FF33', '#33FFE3', '#3399FF','#9033FF']

# Reclassify the map
result = result.remap([0, 1, 2, 3, 4,5,6,7,8,9], [1, 2, 3, 4, 5,6,7,8,9,10])

Map.addLayer(
    result, {'min': 1, 'max': 10, 'palette': legend_colors}, 'Labelled clusters'
)
Map.add_legend(
    legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright'
)
Map

Map(bottom=421558.55558538437, center=[33.31832027263439, -7.695358058759015], controls=(WidgetControl(options…

### Visualize the result

In [ ]:
print('Change layer opacity:')
cluster_layer = Map.layers[-2]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

### Export the result

Export the result directly to your computer:

In [ ]:
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'cluster.tif')

In [ ]:
geemap.ee_export_image(result, filename=out_file, scale=90)

Export the result to Google Drive:

In [ ]:
geemap.ee_export_image_to_drive(
    result, description='clusters', folder='export', scale=90
)